# Implementation of LSTM to Motion Classification Application
Implementation of LSTM technique to motion detection application. Developed by James Youngchae Chee.

### 1. Setting Up Environment

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 2. Importing Data
This dataset contains fall, swing and still gyro and accelerometer data.

In [2]:
train_data = [pd.read_csv('./data/swing.csv',header=None),
             pd.read_csv('./data/still.csv',header=None)]
test_data = [pd.read_csv('./data/swingtest.csv',header=None),
             pd.read_csv('./data/stilltest.csv',header=None)]

### 3. Examining Data
Each Data has <b>1200</b> dimensions --> Stacked numerous sensor data together
<br>Each Retrieval of sesor data looks like (linear_accel x,y,z, Gyro x,y,z)
<br><b>Number of train_data : 50
<br><b>Number of test_data : 10
    
### Need Better dataset  --> each action data is 10 seconds long which doesn't represents a reliable labeled data

In [3]:
train_data[0].head(2)

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.016617,0.358795,-1.035721,-0.004248,0.113635,0.046729,-1.263992,0.626968,0.064026,-0.002124,...,3.639145,0.027612,2.485107,0.805005,0.223007,-1.093353,3.568604,0.00000,2.291821,0.781641
1,2.439789,-0.589111,1.827896,-0.011682,-0.004248,0.007434,2.439789,-0.589111,1.827896,-0.011682,...,1.446808,-0.045667,1.305212,0.451355,2.819183,-0.452820,0.934021,-0.04248,0.881470,0.316480


In [4]:
train_data[1].head(2)

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,-3.282227,-1.110794,0.673569,3.143555,-2.313062,-0.727478,-3.314667,-1.153244,0.662994,0.104077,...,0.044067,-0.002124,-0.005310,0.004248,-0.001617,0.042175,0.044067,-0.013806,-0.007434,0.003186
1,-0.064117,0.273438,0.307724,-0.023364,-0.062659,0.046729,-7.253113,4.420563,8.765472,-0.192224,...,-0.033951,0.015930,0.012744,-0.011682,-0.038284,0.046951,0.061691,0.031860,-0.066907,0.000000


In [5]:
train_data[0].shape

(50, 1200)

In [6]:
test_data[0].shape

(10, 1200)

### 4. Define Model

In [7]:
# Model Representation
#                        O * W + b -> 2 labels for each data, O[? 6], W[6 2], B[2]
#                       ^ (O: output 6 vec from 6 vec input)
#                       |
#      +-+  +-+       +--+
#      |1|->|2|-> ... |200| time_step_size = 200
#      +-+  +-+       +--+
#       ^    ^    ...  ^
#       |    |         |
#data1:[6]  [6]  ...  [6] input_vec_size = lstm_size = 6
#data2:[6]  [6]  ...  [6]
# ...

# configuration variables
EPOCHS = 10000

input_vec_size = lstm_size = 6
time_step_size = 200

BATCH_SIZE = 5 # training batch size
test_size = 10 # validation batch size

def model(X,W,B,lstm_size):
    # X, input shape: (batch_size, input_vec_size, time_step_size)
    # XT, shape: (time_step_size,batch_size,input_vec_size)
    XT = tf.transpose(X,[1,2,0])
    # XR, shape: (time_step_size * batch_size , input_vec_size)
    XR = tf.reshape(XT,[-1,input_vec_size])
    # X_time, each data shape: (batch_size,input_vec_size)
    X_time = tf.split(XR,time_step_size,0)
    
    # make lstm cell with input_vec_size
    lstm = rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=True)
    
    # Get lstm cell output, time_step_size (200) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = rnn.static_rnn(lstm, X_time, dtype=tf.float32)

    return tf.matmul(outputs[-1], W) + B, lstm.state_size

### 5. Reshape input data and make them into Tensorflow Dataset

In [8]:
trX = np.reshape(np.array([train_data[i].values for i in range(2)]),(100,1200)) # converge two classes of data
teX = np.reshape(np.array([test_data[i].values for i in range(2)]),(20,1200)) # converge two classes of data
trX = np.transpose(np.reshape(trX,(-1,time_step_size,input_vec_size)),[0,2,1]) # reshape trX (-1,input_vec_size,time_step_size)
teX = np.transpose(np.reshape(teX,(-1,time_step_size,input_vec_size)),[0,2,1]) # reshape teX (-1,input_vec_size,time_step_size)
trY = np.append(np.array([[1,0] for _ in range(50)]),np.array([[0,1] for _ in range(50)]),axis=0)
teY = np.append(np.array([[1,0] for _ in range(10)]),np.array([[0,1] for _ in range(10)]),axis=0)

In [9]:
print('trX: ',trX.shape)
print('trY: ',trY.shape)
print('teX: ',teX.shape)
print('teY: ',teY.shape)

('trX: ', (100, 6, 200))
('trY: ', (100, 2))
('teX: ', (20, 6, 200))
('teY: ', (20, 2))


In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((trX,trY)).shuffle(buffer_size=100).repeat().batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((teX,teY)).shuffle(buffer_size=100).repeat().batch(test_size)

### 6. Define Cost Function, Variables and Placeholders

In [11]:
iter = train_dataset.make_initializable_iterator()
train_x, train_y = iter.get_next()
train_x = tf.cast(train_x,tf.float32)

test_iter = test_dataset.make_initializable_iterator()
test_x, test_y = iter.get_next()
test_x = tf.cast(test_x,tf.float32)

X = tf.placeholder(tf.float32, [None, 6, 200])
Y = tf.placeholder(tf.float32, [None, 2])

# weight : input_vec_size  --> output 2 labels
W = tf.Variable(tf.random_normal([input_vec_size, 2], stddev=0.01))
B = tf.Variable(tf.random_normal([2], stddev=0.01))

py_x, state_size = model(X, W, B, lstm_size)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x,axis = 1)
accuracy_op = tf.metrics.accuracy(labels = tf.argmax(Y, axis = 1),predictions = predict_op)

### 7. Train

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(iter.initializer)
    sess.run(test_iter.initializer)
    for i in range(EPOCHS):
        x_tr,y_tr,x_te,y_te = sess.run([train_x,train_y,test_x,test_y])
        _, loss_value = sess.run([train_op, cost],feed_dict={X:x_tr,Y:y_tr})
        accuracy = sess.run(accuracy_op,feed_dict={X:x_te,Y:y_te})
        print("Iter: ",i,", Loss: ",loss_value,", Accuracy: ",accuracy[0])

('Iter: ', 0, ', Loss: ', 0.6945225, ', Accuracy: ', 0.0)
('Iter: ', 1, ', Loss: ', 0.69016868, ', Accuracy: ', 0.60000002)
('Iter: ', 2, ', Loss: ', 0.69680309, ', Accuracy: ', 0.5)
('Iter: ', 3, ', Loss: ', 0.69768792, ', Accuracy: ', 0.60000002)
('Iter: ', 4, ', Loss: ', 0.69459701, ', Accuracy: ', 0.60000002)
('Iter: ', 5, ', Loss: ', 0.69828475, ', Accuracy: ', 0.56)
('Iter: ', 6, ', Loss: ', 0.68968642, ', Accuracy: ', 0.56666666)
('Iter: ', 7, ', Loss: ', 0.69429886, ', Accuracy: ', 0.62857145)
('Iter: ', 8, ', Loss: ', 0.69241846, ', Accuracy: ', 0.60000002)
('Iter: ', 9, ', Loss: ', 0.69355875, ', Accuracy: ', 0.5777778)
('Iter: ', 10, ', Loss: ', 0.69423807, ', Accuracy: ', 0.57999998)
('Iter: ', 11, ', Loss: ', 0.69441533, ', Accuracy: ', 0.58181816)
('Iter: ', 12, ', Loss: ', 0.69672173, ', Accuracy: ', 0.58333331)
('Iter: ', 13, ', Loss: ', 0.69340467, ', Accuracy: ', 0.56923079)
('Iter: ', 14, ', Loss: ', 0.69496489, ', Accuracy: ', 0.5714286)
('Iter: ', 15, ', Loss: ', 0

('Iter: ', 127, ', Loss: ', 0.6855129, ', Accuracy: ', 0.56377953)
('Iter: ', 128, ', Loss: ', 0.69204193, ', Accuracy: ', 0.56406248)
('Iter: ', 129, ', Loss: ', 0.69039124, ', Accuracy: ', 0.56434107)
('Iter: ', 130, ', Loss: ', 0.68596995, ', Accuracy: ', 0.56615382)
('Iter: ', 131, ', Loss: ', 0.67811763, ', Accuracy: ', 0.56641221)
('Iter: ', 132, ', Loss: ', 0.69387972, ', Accuracy: ', 0.56818181)
('Iter: ', 133, ', Loss: ', 0.69304359, ', Accuracy: ', 0.56842107)
('Iter: ', 134, ', Loss: ', 0.66919112, ', Accuracy: ', 0.56865674)
('Iter: ', 135, ', Loss: ', 0.69082844, ', Accuracy: ', 0.5688889)
('Iter: ', 136, ', Loss: ', 0.67041314, ', Accuracy: ', 0.56911767)
('Iter: ', 137, ', Loss: ', 0.69134653, ', Accuracy: ', 0.56934309)
('Iter: ', 138, ', Loss: ', 0.66896087, ', Accuracy: ', 0.56956524)
('Iter: ', 139, ', Loss: ', 0.69090712, ', Accuracy: ', 0.56834531)
('Iter: ', 140, ', Loss: ', 0.69132817, ', Accuracy: ', 0.56857145)
('Iter: ', 141, ', Loss: ', 0.67548275, ', Accurac

('Iter: ', 250, ', Loss: ', 0.56838745, ', Accuracy: ', 0.62239999)
('Iter: ', 251, ', Loss: ', 0.54445153, ', Accuracy: ', 0.62390441)
('Iter: ', 252, ', Loss: ', 0.53886545, ', Accuracy: ', 0.62539685)
('Iter: ', 253, ', Loss: ', 0.79371178, ', Accuracy: ', 0.62687749)
('Iter: ', 254, ', Loss: ', 0.60201162, ', Accuracy: ', 0.62755907)
('Iter: ', 255, ', Loss: ', 0.5261175, ', Accuracy: ', 0.62901962)
('Iter: ', 256, ', Loss: ', 0.59962666, ', Accuracy: ', 0.63046873)
('Iter: ', 257, ', Loss: ', 0.59349716, ', Accuracy: ', 0.63190663)
('Iter: ', 258, ', Loss: ', 0.57676208, ', Accuracy: ', 0.63333333)
('Iter: ', 259, ', Loss: ', 0.5087043, ', Accuracy: ', 0.63397682)
('Iter: ', 260, ', Loss: ', 0.50374949, ', Accuracy: ', 0.63461536)
('Iter: ', 261, ', Loss: ', 0.55929101, ', Accuracy: ', 0.63524902)
('Iter: ', 262, ', Loss: ', 0.64599997, ', Accuracy: ', 0.63587785)
('Iter: ', 263, ', Loss: ', 0.57980269, ', Accuracy: ', 0.63726234)
('Iter: ', 264, ', Loss: ', 0.55999708, ', Accurac

('Iter: ', 373, ', Loss: ', 0.51533425, ', Accuracy: ', 0.72868633)
('Iter: ', 374, ', Loss: ', 0.32401982, ', Accuracy: ', 0.72941178)
('Iter: ', 375, ', Loss: ', 0.51498687, ', Accuracy: ', 0.73013335)
('Iter: ', 376, ', Loss: ', 0.67800438, ', Accuracy: ', 0.73085105)
('Iter: ', 377, ', Loss: ', 0.34649751, ', Accuracy: ', 0.731565)
('Iter: ', 378, ', Loss: ', 0.3477897, ', Accuracy: ', 0.73227513)
('Iter: ', 379, ', Loss: ', 0.32571918, ', Accuracy: ', 0.7329815)
('Iter: ', 380, ', Loss: ', 0.30413395, ', Accuracy: ', 0.73315787)
('Iter: ', 381, ', Loss: ', 0.32543322, ', Accuracy: ', 0.73385829)
('Iter: ', 382, ', Loss: ', 0.30733308, ', Accuracy: ', 0.73455495)
('Iter: ', 383, ', Loss: ', 0.28362551, ', Accuracy: ', 0.73472583)
('Iter: ', 384, ', Loss: ', 0.32734653, ', Accuracy: ', 0.73489583)
('Iter: ', 385, ', Loss: ', 0.30047333, ', Accuracy: ', 0.73558444)
('Iter: ', 386, ', Loss: ', 0.28479594, ', Accuracy: ', 0.73626941)
('Iter: ', 387, ', Loss: ', 0.33179039, ', Accuracy:

('Iter: ', 497, ', Loss: ', 0.16678354, ', Accuracy: ', 0.78390342)
('Iter: ', 498, ', Loss: ', 0.17379561, ', Accuracy: ', 0.78433734)
('Iter: ', 499, ', Loss: ', 0.29200128, ', Accuracy: ', 0.78476954)
('Iter: ', 500, ', Loss: ', 0.1617792, ', Accuracy: ', 0.7852)
('Iter: ', 501, ', Loss: ', 0.43205649, ', Accuracy: ', 0.78562874)
('Iter: ', 502, ', Loss: ', 0.13807258, ', Accuracy: ', 0.7860558)
('Iter: ', 503, ', Loss: ', 0.1843842, ', Accuracy: ', 0.78648114)
('Iter: ', 504, ', Loss: ', 0.13841556, ', Accuracy: ', 0.78690475)
('Iter: ', 505, ', Loss: ', 0.45434412, ', Accuracy: ', 0.78732675)
('Iter: ', 506, ', Loss: ', 0.18270752, ', Accuracy: ', 0.78774703)
('Iter: ', 507, ', Loss: ', 0.1920121, ', Accuracy: ', 0.78816569)
('Iter: ', 508, ', Loss: ', 0.12621053, ', Accuracy: ', 0.78858268)
('Iter: ', 509, ', Loss: ', 0.18623538, ', Accuracy: ', 0.78899801)
('Iter: ', 510, ', Loss: ', 0.14928988, ', Accuracy: ', 0.78941178)
('Iter: ', 511, ', Loss: ', 0.1497018, ', Accuracy: ', 0

...

('Iter: ', 9526, ', Loss: ', 1.4543519e-06, ', Accuracy: ', 0.98528236)
('Iter: ', 9527, ', Loss: ', 9.2983203e-07, ', Accuracy: ', 0.98528391)
('Iter: ', 9528, ', Loss: ', 7.3909729e-07, ', Accuracy: ', 0.98528546)
('Iter: ', 9529, ', Loss: ', 1.0728829e-06, ', Accuracy: ', 0.98528701)
('Iter: ', 9530, ', Loss: ', 2.9325438e-06, ', Accuracy: ', 0.98528856)
('Iter: ', 9531, ', Loss: ', 9.2983203e-07, ', Accuracy: ', 0.98529011)
('Iter: ', 9532, ', Loss: ', 0.035199292, ', Accuracy: ', 0.98529166)
('Iter: ', 9533, ', Loss: ', 2.2649733e-06, ', Accuracy: ', 0.98529321)
('Iter: ', 9534, ', Loss: ', 2.0265556e-06, ', Accuracy: ', 0.98529476)
('Iter: ', 9535, ', Loss: ', 4.2915332e-07, ', Accuracy: ', 0.98529625)
('Iter: ', 9536, ', Loss: ', 1.6927701e-06, ', Accuracy: ', 0.9852978)
('Iter: ', 9537, ', Loss: ', 7.3909723e-07, ', Accuracy: ', 0.98529935)
('Iter: ', 9538, ', Loss: ', 9.7751558e-07, ', Accuracy: ', 0.9853009)
('Iter: ', 9539, ', Loss: ', 3.7431619e-06, ', Accuracy: ', 0.985302

('Iter: ', 9644, ', Loss: ', 2.2649738e-06, ', Accuracy: ', 0.98546249)
('Iter: ', 9645, ', Loss: ', 1.311301e-06, ', Accuracy: ', 0.98546398)
('Iter: ', 9646, ', Loss: ', 1.0013575e-06, ', Accuracy: ', 0.98546547)
('Iter: ', 9647, ', Loss: ', 2.1457649e-06, ', Accuracy: ', 0.98546696)
('Iter: ', 9648, ', Loss: ', 2.4318663e-06, ', Accuracy: ', 0.98546851)
('Iter: ', 9649, ', Loss: ', 4.8160405e-06, ', Accuracy: ', 0.98547)
('Iter: ', 9650, ', Loss: ', 1.8596629e-06, ', Accuracy: ', 0.98547149)
('Iter: ', 9651, ', Loss: ', 2.3364978e-06, ', Accuracy: ', 0.98547304)
('Iter: ', 9652, ', Loss: ', 1.1205667e-06, ', Accuracy: ', 0.98547453)
('Iter: ', 9653, ', Loss: ', 9.0599013e-07, ', Accuracy: ', 0.98547602)
('Iter: ', 9654, ', Loss: ', 2.384181e-06, ', Accuracy: ', 0.98547751)
('Iter: ', 9655, ', Loss: ', 6.9141362e-07, ', Accuracy: ', 0.985479)
('Iter: ', 9656, ', Loss: ', 7.3909735e-07, ', Accuracy: ', 0.98548055)
('Iter: ', 9657, ', Loss: ', 1.6450867e-06, ', Accuracy: ', 0.98548204)

('Iter: ', 9760, ', Loss: ', 1.0967248e-06, ', Accuracy: ', 0.98559427)
('Iter: ', 9761, ', Loss: ', 7.6293912e-07, ', Accuracy: ', 0.98559576)
('Iter: ', 9762, ', Loss: ', 1.4543517e-06, ', Accuracy: ', 0.98559719)
('Iter: ', 9763, ', Loss: ', 2.0503974e-06, ', Accuracy: ', 0.98559868)
('Iter: ', 9764, ', Loss: ', 3.0279109e-06, ', Accuracy: ', 0.98560017)
('Iter: ', 9765, ', Loss: ', 8.3446469e-07, ', Accuracy: ', 0.98560166)
('Iter: ', 9766, ', Loss: ', 1.4781939e-06, ', Accuracy: ', 0.98560309)
('Iter: ', 9767, ', Loss: ', 1.7881372e-06, ', Accuracy: ', 0.98560458)
('Iter: ', 9768, ', Loss: ', 5.4836255e-07, ', Accuracy: ', 0.98560607)
('Iter: ', 9769, ', Loss: ', 1.0251993e-06, ', Accuracy: ', 0.9856075)
('Iter: ', 9770, ', Loss: ', 1.8596631e-06, ', Accuracy: ', 0.98560899)
('Iter: ', 9771, ', Loss: ', 1.4305103e-06, ', Accuracy: ', 0.98561049)
('Iter: ', 9772, ', Loss: ', 1.0013575e-06, ', Accuracy: ', 0.98561198)
('Iter: ', 9773, ', Loss: ', 2.1457647e-06, ', Accuracy: ', 0.985

('Iter: ', 9876, ', Loss: ', 8.5830652e-07, ', Accuracy: ', 0.98572296)
('Iter: ', 9877, ', Loss: ', 5.7220439e-07, ', Accuracy: ', 0.98572439)
('Iter: ', 9878, ', Loss: ', 6.9141362e-07, ', Accuracy: ', 0.98572588)
('Iter: ', 9879, ', Loss: ', 1.9788722e-06, ', Accuracy: ', 0.98572731)
('Iter: ', 9880, ', Loss: ', 1.3113009e-06, ', Accuracy: ', 0.98572874)
('Iter: ', 9881, ', Loss: ', 8.3446469e-07, ', Accuracy: ', 0.98573017)
('Iter: ', 9882, ', Loss: ', 9.5367386e-07, ', Accuracy: ', 0.98573166)
('Iter: ', 9883, ', Loss: ', 1.6689299e-07, ', Accuracy: ', 0.98573309)
('Iter: ', 9884, ', Loss: ', 5.0067894e-07, ', Accuracy: ', 0.98573452)
('Iter: ', 9885, ', Loss: ', 1.5258777e-06, ', Accuracy: ', 0.98573595)
('Iter: ', 9886, ', Loss: ', 1.2397759e-06, ', Accuracy: ', 0.98573738)
('Iter: ', 9887, ', Loss: ', 1.4781933e-06, ', Accuracy: ', 0.98573887)
('Iter: ', 9888, ', Loss: ', 1.6689286e-06, ', Accuracy: ', 0.9857403)
('Iter: ', 9889, ', Loss: ', 1.8119794e-06, ', Accuracy: ', 0.985

('Iter: ', 9993, ', Loss: ', 6.1988806e-07, ', Accuracy: ', 0.98587012)
('Iter: ', 9994, ', Loss: ', 8.3446469e-07, ', Accuracy: ', 0.98587149)
('Iter: ', 9995, ', Loss: ', 3.5762778e-07, ', Accuracy: ', 0.98587292)
('Iter: ', 9996, ', Loss: ', 1.9311881e-06, ', Accuracy: ', 0.98587435)
('Iter: ', 9997, ', Loss: ', 6.6757173e-07, ', Accuracy: ', 0.98587579)
('Iter: ', 9998, ', Loss: ', 3.1232782e-06, ', Accuracy: ', 0.98587716)
('Iter: ', 9999, ', Loss: ', 8.3446469e-07, ', Accuracy: ', 0.98587859)
